### Joel Greenblat's Magic Formula 

In [1]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import requests

In [2]:
# Extrai o banco de dados da página 'Fundamentus'
url     = ('https://www.fundamentus.com.br/resultado.php')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
formData = {
    'pl_min': '',
    'pl_max': '',
    'pvp_min': '',
    'pvp_max': '',
    'psr_min': '',
    'psr_max': '',
    'divy_min': '',
    'divy_max': '',
    'pativos_min': '',
    'pativos_max': '',
    'pcapgiro_min': '',
    'pcapgiro_max': '',
    'pebit_min': '',
    'pebit_max': '',
    'fgrah_min': '',
    'fgrah_max': '',
    'firma_ebit_min': '',
    'firma_ebit_max': '',
    'firma_ebitda_min': '',
    'firma_ebitda_max': '',
    'margemebit_min': '',
    'margemebit_max': '',
    'margemliq_min': '',
    'margemliq_max': '',
    'liqcorr_min': '',
    'liqcorr_max': '',
    'roic_min': '',
    'roic_max': '',
    'roe_min': '',
    'roe_max': '',
    'liq_min': '',
    'liq_max': '',
    'patrim_min': '',
    'patrim_max': '',
    'divbruta_min': '',
    'divbruta_max': '',
    'tx_cresc_rec_min': '',
    'tx_cresc_rec_max': '',
    'setor': '',
    'negociada': 'ON',
    'ordem': '1',
    'x': '35',
    'y': '20',
}
# res     = requests.get(url, headers=headers, data=formData)
res     = requests.post(url, headers=headers, data=formData)
df      = pd.read_html(res.content)[0]
df.head(5)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,GFSA3,303,-65180.0,58,810,"0,00%",231,79.00000,1007.0,266,...,1620.0,"8,04%","-0,10%",175,"2,65%","-0,09%",20.086.70000,1.630.560.00000,78,"18,80%"
1,DASA3,4480,-40643.0,338,2645,"0,56%",1381,853.00000,59499.0,-844,...,3005.0,"0,44%","-0,56%",161,"0,33%","-0,83%",9.684.86000,7.265.200.00000,79,"27,54%"
2,MAPT4,4679,-39091.0,-732,0,"0,00%",55627,-3.90909,-45745.0,-647,...,-45745.0,"0,00%","0,00%",0,"-12,34%","1,87%",3.65756,-5.876.00000,0,"0,00%"
3,WEST3,593,-26933.0,162,2392,"0,00%",1229,185.00000,-2740.0,203,...,-1582.0,"-8,73%","-0,89%",462,"-23,24%","-0,60%",5.396.76000,403.271.00000,8,"0,00%"
4,SBFG3,3368,-26664.0,431,2275,"0,00%",1235,644.00000,6781.0,-580,...,2519.0,"3,35%","-0,85%",162,"2,20%","-1,62%",27.842.80000,1.898.800.00000,118,"30,19%"


In [3]:
# Converte coluna ROIC de valores string para float
df['ROIC'] = df['ROIC'].str.replace(',','.').str.rstrip('%').astype('float');

In [4]:
# Seleciona valores maiores que 0
selecao = (df['EV/EBIT'] > 0) & (df['ROIC'] > 0) 
df2 = df[selecao].copy(deep=True)

In [5]:
# RANK EV/EBIT
df2.sort_values(by="EV/EBIT", ascending=True, inplace=True) # Sort Crescente
df2['Rank_EV'] = range(1,df2.shape[0]+1) # insere os indices do Rank

df2[['Papel','EV/EBIT','Rank_EV']].head(5)

,Papel,EV/EBIT,Rank_EV
84,HETA4,1.01444,1
77,HETA3,1.14233,2
277,PSSA3,48.00000,3
390,MNPR3,175.00000,4
156,GOAU3,176.00000,5


In [6]:
# RANK ROIC
df2.sort_values(by="ROIC", ascending=False, inplace=True) # Sort Decrescente
df2['Rank_ROIC'] = range(1,df2.shape[0]+1) # Insere os Indices do Rank

df2[['Papel','ROIC','Rank_ROIC']].head(5)

,Papel,ROIC,Rank_ROIC
254,ATOM3,82.04,1
163,CMIN3,81.03,2
256,WIZS3,68.27,3
335,WHRL4,55.87,4
304,WHRL3,55.87,5


In [7]:
# RANK Magic
df2['Rank_Soma'] = df2['Rank_ROIC'] + df2['Rank_EV'] # Somatório dos Ranks

df2 = df2.sort_values(by="Rank_Soma") # Sort dos Ranks

df2['Rank_Magic'] = range(1,df2.shape[0]+1) # Insere indices

selecao = ['Papel','EV/EBIT','Rank_EV','ROIC','Rank_ROIC','Rank_Soma','Rank_Magic'] # Seleciona apenas colunas

df2[selecao].head(20) # 20 primeiros papeis

,Papel,EV/EBIT,Rank_EV,ROIC,Rank_ROIC,Rank_Soma,Rank_Magic
163,CMIN3,234.0,7,81.03,2,9,1
277,PSSA3,48.0,3,54.98,6,9,2
181,VALE3,279.0,13,44.97,9,22,3
177,PLPL3,261.0,12,41.15,13,25,4
133,CSNA3,292.0,14,40.20,14,28,5
256,WIZS3,401.0,28,68.27,3,31,6
136,BRKM6,246.0,8,33.38,28,36,7
142,TASA4,442.0,33,49.06,7,40,8
148,ETER3,385.0,26,39.90,15,41,9
146,TASA3,456.0,34,49.06,8,42,10


In [8]:
# Tabela contendo os setores de cada papel
df_setorial = pd.read_csv('setorial.csv')

df_setorial.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
2,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",DOMMO,DMMO,NaN
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
4,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PET MANGUINH,RPMG,NaN


In [9]:
# Remove linhas vazias (problema gerado na hora de crirar o csv e corrigido aqui)
df_setorial2 = df_setorial[df_setorial["SEGMENTO.1"].notna()]

df_setorial2.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
5,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS,PETR,N2
6,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS BR,BRDT,NM


In [10]:
df6 = df5.copy() # Copia do df5
df6['Papel'] = df5['Papel'].str.rstrip('1234567890') # Pega somente Letras do papel
df6['Codigo'] = df5['Papel'].str.extract('(\d)', expand=True).astype('int') # Separa o Tipo/Código em outra coluna (ordinária, preferencial, etc..)
df6.head(5)

NameError: name 'df5' is not defined

In [ ]:
# Próximo passo: PUXAR O SETOR DO df_setorial2 com base no 'Papel' do df6